# Example to train a model
Using the openstf tasks

In [1]:
import pandas as pd
import IPython
from openstef.pipeline.train_model import train_model_pipeline
from openstef.pipeline.create_forecast import create_forecast_pipeline
from openstef.metrics import metrics

C:\Users\AL20611\Anaconda3\envs\research\lib\site-packages\xgboost\compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


2022-03-18 13:28.16 [warning  ] Proloaf not available, switching to xgboost! See Readme how to install proloaf dependencies


In [2]:
# define properties of training/prediction. We call this a 'prediction_job'
pj=dict(id=287,
        model='xgb',
        quantiles=[0.05, 0.1, 0.3, 0.5, 0.7, 0.9, 0.95],
        name="backtest",
        forecast_type="demand",
        description="description",
        resolution_minutes=15,
        hyper_params={}, # Note, this should become optional
        feature_names=None, # Note, this should become optional
       )

# Load input data
train_data = pd.read_csv('C:\\repos\\icarus-analyses\\analyses\\moeilijke_cases\\moeilijke_case_Oosterwolde_train_data.csv', index_col=0, parse_dates=True)
to_forecast_data = pd.read_csv('C:\\repos\\icarus-analyses\\analyses\\moeilijke_cases\\moeilijke_case_Oosterwolde_test_data.csv', index_col=0, parse_dates=True)


In [3]:
train_data.head()

,load,clouds,radiation,temp,winddeg,windspeed,windspeed_100m,pressure,humidity,mxlD,...,sjv_E1A,sjv_E1B,sjv_E1C,sjv_E2A,sjv_E2B,sjv_E3A,sjv_E3B,sjv_E3C,sjv_E3D,sjv_E4A
2021-10-26 00:00:00+00:00,9.573333,99.774170,5.588182e-11,9.323700,202.050003,4.837466,8.647599,101597.250000,0.952319,378.603333,...,0.000019,0.000020,0.000021,0.000024,0.000025,0.000016,0.000019,0.000019,0.000025,0.00006
2021-10-26 00:15:00+00:00,9.520000,76.757050,5.588227e-11,8.956253,212.332508,5.058731,8.762285,101594.460938,0.957519,386.540833,...,0.000018,0.000019,0.000020,0.000024,0.000025,0.000016,0.000019,0.000019,0.000025,0.00006
2021-10-26 00:30:00+00:00,9.550000,53.739929,5.588272e-11,8.588806,222.615013,5.279995,8.876970,101591.671875,0.962718,394.478333,...,0.000018,0.000019,0.000019,0.000024,0.000024,0.000016,0.000019,0.000019,0.000026,0.00006
2021-10-26 00:45:00+00:00,9.530000,30.722809,5.588362e-11,8.221359,232.897518,5.501260,8.991656,101588.882812,0.967917,402.415833,...,0.000017,0.000019,0.000019,0.000024,0.000025,0.000016,0.000019,0.000019,0.000026,0.00006
2021-10-26 01:00:00+00:00,9.423333,7.705688,5.588453e-11,7.853912,243.180023,5.722525,9.106341,101586.093750,0.973117,410.353333,...,0.000017,0.000018,0.000018,0.000023,0.000024,0.000016,0.000019,0.000019,0.000026,0.00006


In [4]:
to_forecast_data.head()

,load,clouds,radiation,temp,winddeg,windspeed,windspeed_100m,pressure,humidity,mxlD,...,sjv_E1A,sjv_E1B,sjv_E1C,sjv_E2A,sjv_E2B,sjv_E3A,sjv_E3B,sjv_E3C,sjv_E3D,sjv_E4A
2022-02-23 00:00:00+00:00,10.966667,0.0,2.790956e-11,2.572968,137.576294,9.086503,15.218234,101848.171875,0.876644,853.704224,...,0.000026,0.000028,0.000027,0.000026,0.000027,0.00002,0.000022,0.000022,0.000025,0.00006
2022-02-23 00:15:00+00:00,10.866667,0.0,4.186433e-11,2.484604,139.713455,9.103703,15.249041,101871.111328,0.882332,837.125107,...,0.000025,0.000027,0.000026,0.000025,0.000026,0.00002,0.000022,0.000022,0.000025,0.00006
2022-02-23 00:30:00+00:00,10.673333,0.0,5.581911e-11,2.396240,141.850616,9.120903,15.279848,101894.050781,0.888019,820.545990,...,0.000023,0.000025,0.000024,0.000025,0.000026,0.00002,0.000022,0.000022,0.000025,0.00006
2022-02-23 00:45:00+00:00,10.656667,0.0,5.581912e-11,2.307877,143.987778,9.138102,15.310655,101916.990234,0.893707,803.966873,...,0.000022,0.000024,0.000023,0.000024,0.000026,0.00002,0.000022,0.000022,0.000025,0.00006
2022-02-23 01:00:00+00:00,10.626667,0.0,5.581912e-11,2.219513,146.124939,9.155302,15.341462,101939.929688,0.899395,787.387756,...,0.000022,0.000024,0.000022,0.000024,0.000025,0.00002,0.000022,0.000022,0.000025,0.00006


# Train a model
Train a model using the high-level pipeline. Store the model and reports on training proces in ./trained_models

In [5]:
models = train_model_pipeline(
    pj,
    train_data,
    check_old_model_age=False,
    trained_models_folder='./trained_models',
    )

2022-03-18 13:28.16 [debug    ] MLflow path at init= file:///C:/repos/openstef-offline-example/examples/trained_models/mlruns
2022-03-18 13:28.17 [info     ] Model successfully loaded with MLflow


c:\repos\openstef\openstef\feature_engineering\feature_applicator.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
c:\repos\openstef\openstef\feature_engineering\feature_applicator.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(res)
C:\Users\AL20611\Anaconda3\envs\research\lib\site-packages\numpy\lib\arraysetops.py:608: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  mask |= (ar1 == a)
c:\repos\openstef\openstef\model_selection\model_selection.py:226: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard l

2022-03-18 13:28.22 [info     ] New model is better than old model, continuing with training procces


C:\Users\AL20611\Anaconda3\envs\research\lib\site-packages\xgboost\data.py:192: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



2022-03-18 13:28.26 [info     ] Model saved with MLflow        pid=287
2022-03-18 13:28.28 [info     ] logged figures to MLflow
2022-03-18 13:28.28 [debug    ] MLflow path after saving= file:///C:/repos/openstef-offline-example/examples/trained_models/mlruns
2022-03-18 13:28.29 [info     ] Stored report to disk: trained_models\287
2022-03-18 13:28.29 [debug    ] Going to delete old models. 11>10
2022-03-18 13:28.29 [debug    ] Going to remove run fb4d9c5499fa4de88f90bb1236e3c90b, from 2022-03-15 14:09:22.930000+00:00. Artifact location: ./trained_models\mlruns/1/fb4d9c5499fa4de88f90bb1236e3c90b
2022-03-18 13:28.29 [debug    ] Removed run


You can find the trained model in ./trained_models, along with reports on the training process

In [6]:
## Inspect local files
IPython.display.HTML(f"<iframe src=./trained_models/{pj['id']}/Predictor0.25.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/Predictor47.0.html width=800 height=400></iframe>"
                     f"<iframe src=./trained_models/{pj['id']}/weight_plot.html width=800 height=400></iframe>")

C:\Users\AL20611\Anaconda3\envs\research\lib\site-packages\IPython\core\display.py:724: UserWarning:

Consider using IPython.display.IFrame instead



In [7]:
import numpy as np
realised = to_forecast_data['load'].copy(deep=True)
to_forecast_data['load'] = np.nan

# Make a forecast
forecast = create_forecast_pipeline(pj, to_forecast_data,trained_models_folder='./trained_models')

2022-03-18 13:28.29 [debug    ] MLflow path at init= file:///C:/repos/openstef-offline-example/examples/trained_models/mlruns
2022-03-18 13:28.29 [info     ] Model successfully loaded with MLflow
2022-03-18 13:28.29 [info     ] Found 481 values of constant load (repeated values), converted to NaN value. cleansing_step=repeated_values frac_values=1.0 num_values=481 pj_id=287
2022-03-18 13:28.30 [info     ] Postproces in preparation of storing


C:\Users\AL20611\Anaconda3\envs\research\lib\site-packages\xgboost\data.py:192: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



In [13]:
import cufflinks
cufflinks.go_offline()
forecast['load'] = realised
forecast[['load','forecast' ]].iplot()

In [9]:
forecast

,forecast,tAhead,stdev,quantile_P05,quantile_P10,quantile_P30,quantile_P50,quantile_P70,quantile_P90,quantile_P95,pid,customer,description,type,algtype,realised
2022-02-23 00:00:00+00:00,11.097845,-564.50,-7.862515e+19,1.293269e+20,1.007622e+20,4.123107e+19,11.097845,-4.123107e+19,-1.007622e+20,-1.293269e+20,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.966667
2022-02-23 00:15:00+00:00,11.097845,-564.25,-7.696995e+19,1.266043e+20,9.864096e+19,4.036308e+19,11.097845,-4.036308e+19,-9.864096e+19,-1.266043e+20,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.866667
2022-02-23 00:30:00+00:00,10.850774,-564.00,-7.534959e+19,1.239390e+20,9.656438e+19,3.951336e+19,10.850774,-3.951336e+19,-9.656438e+19,-1.239390e+20,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.673333
2022-02-23 00:45:00+00:00,10.850774,-563.75,-7.376334e+19,1.213299e+20,9.453152e+19,3.868153e+19,10.850774,-3.868153e+19,-9.453152e+19,-1.213299e+20,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.656667
2022-02-23 01:00:00+00:00,10.850774,-563.50,-8.230574e+19,1.353809e+20,1.054790e+20,4.316117e+19,10.850774,-4.316117e+19,-1.054790e+20,-1.353809e+20,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.626667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-27 23:00:00+00:00,11.699283,-445.50,-1.570572e+15,2.583360e+15,2.012768e+15,8.236085e+14,11.699283,-8.236085e+14,-2.012768e+15,-2.583360e+15,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.136667
2022-02-27 23:15:00+00:00,11.699283,-445.25,-1.537508e+15,2.528976e+15,1.970396e+15,8.062700e+14,11.699283,-8.062700e+14,-1.970396e+15,-2.528976e+15,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,10.003333
2022-02-27 23:30:00+00:00,10.861599,-445.00,-1.505141e+15,2.475736e+15,1.928915e+15,7.892966e+14,10.861599,-7.892966e+14,-1.928915e+15,-2.475736e+15,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,9.663333
2022-02-27 23:45:00+00:00,10.738637,-444.75,-1.473455e+15,2.423617e+15,1.888308e+15,7.726804e+14,10.738637,-7.726804e+14,-1.888308e+15,-2.423617e+15,287,backtest,description,demand,/C:/repos/openstef-offline-example/examples/tr...,9.683333


In [25]:
default_metrics ={
    "RMSE": metrics.rmse(forecast["load"], forecast["forecast"]),
    "bias": metrics.bias(forecast["load"], forecast["forecast"]),
    "NSME": metrics.nsme(forecast["load"], forecast["forecast"]),
    "MAE": metrics.mae(forecast["load"], forecast["forecast"]),
    "rMAE": metrics.r_mae(forecast["load"], forecast["forecast"]),
    "rMAE_highest": metrics.r_mae_highest(forecast["load"], forecast["forecast"]),
    "rMNE_highest": metrics.r_mne_highest(forecast["load"], forecast["forecast"]),
    "rMPE_highest": metrics.r_mpe_highest(forecast["load"], forecast["forecast"]),
    "rMAE_lowest": metrics.r_mae_lowest(forecast["load"], forecast["forecast"]),

    "load_range": forecast["load"].max() - forecast["load"].min(),
    "frac_in_1sdev": metrics.frac_in_stdev(forecast["load"], forecast["forecast"], forecast["stdev"]),
    "frac_in_2sdev": metrics.frac_in_stdev(
        forecast["load"], forecast["forecast"], 2 * forecast["stdev"]
    ),
}
pd.DataFrame.from_dict(default_metrics, "index")            

,0
RMSE,12.687696
bias,5.277487
NSME,0.529750
MAE,6.859505
rMAE,0.097142
rMAE_highest,0.091884
rMNE_highest,-0.031401
rMPE_highest,0.000000
rMAE_lowest,0.529763
load_range,70.613333
